In [4]:
import requests
import json
import pandas as pd

###Definig the Function to extract data

In [40]:
def Extract_data (query,tablename):
  # old URL
  url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
  # new URL / after the update 
  #url = 'https://api.thegraph.com/subgraphs/name/ianlapham/v3-minimal'
  frames = []
  for i in range(0,5000,100):
    tmp_query = query%(i)
    r = requests.post(url, json={'query': tmp_query})
    json_data = json.loads(r.text)
    try:
      df_data = json_data['data'][tablename]
      df = pd.DataFrame(df_data)
      frames.append(df)
    except KeyError:
      break

  result = pd.concat(frames)
  return result



#### General Variables

In [41]:
# these token_ids_can be extracted from Token.csv using :uni_id  = token_pd[token_pd["symbol"] == "<symbole of Token>"].iloc[0, 0]
token_usdc_id = "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48"
token_weth_id = "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"
token_wbtc_id = "0x2260fac5e5542a773aa44fbcfedf7c193bc2c599"
token_uni_id = "0x1f9840a85d5af5bf1d1762f925bdaddc4201f984"
token_dai_id = "0x6b175474e89094c44da98b954eedeac495271d0f"

In [1]:
# these token_ids_can be extracted from Pool.csv using :UNI_WETH_3000_id  = pool_df[(pool_df["token0"] == "UNI") & (pool_df["token1"] == "WETH") & (pool_df["feeTier"] == '3000')].iloc[0, 0]
dai_usdc100_id = "0x5777d92f208679db4b9778590fa3cab3ac9e2168"
usdc_WETH_3000_id = "0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8"
wbtc_WETH_3000_id = "0xcbcdf9626bc03e24f779434178a73a0b4bad62ed"
UNI_WETH_3000_id = "0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801"

### Extracting Data

#### Extracting Tokens

In [35]:
#uncomment to see the old code 
# query = """
#  query {
#   tokens(orderBy : derivedETH , orderDirection: desc , skip : %d)
#   {
#     id
#     symbol
#     name
#     decimals
#     totalSupply
#     volume
#     volumeUSD
#     untrackedVolumeUSD
#     feesUSD
#     txCount
#     poolCount
#     totalValueLocked
#     totalValueLockedUSD
#     totalValueLockedUSDUntracked
#     derivedETH
#   }
# }
# """

query = """
query {
  tokens (orderBy:totalValueLockedUSD , orderDirection: desc , skip: %d){
    id
    symbol
    name
    decimals
    totalSupply
    volume
    volumeUSD
    volumeUSDUntracked
    feesUSD
    txCount
    poolCount
    totalValueLocked
    totalValueLockedUSD
    derivedETH
  }
}
"""


In [36]:
 tdf = Extract_data(query, "tokens")
 tdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 99
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   5000 non-null   object
 1   symbol               5000 non-null   object
 2   name                 5000 non-null   object
 3   decimals             5000 non-null   object
 4   totalSupply          5000 non-null   object
 5   volume               5000 non-null   object
 6   volumeUSD            5000 non-null   object
 7   volumeUSDUntracked   5000 non-null   object
 8   feesUSD              5000 non-null   object
 9   txCount              5000 non-null   object
 10  poolCount            5000 non-null   object
 11  totalValueLocked     5000 non-null   object
 12  totalValueLockedUSD  5000 non-null   object
 13  derivedETH           5000 non-null   object
dtypes: object(14)
memory usage: 585.9+ KB


In [37]:
tdf.to_csv('Token.csv',index=False)

#### Extracting TokenDayDatas

In [ ]:
def Extracting_Token_day_data (token_id):
  #old URL
  #url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
  # new URL / after the update 
  url = 'https://api.thegraph.com/subgraphs/name/ianlapham/v3-minimal'
  frames = []
  query = """query {
  tokenDayDatas (where : {token: "%s"}, orderBy: date, orderDirection: desc, skip:%d)
  {
    date
    volumeUSD
    totalValueLockedUSD
    priceUSD
    feesUSD
    open
    high
    low
    close
  }
}"""
  for i in range(0,5000,100):
    tmp_query = query%(token_id,i)
    r = requests.post(url, json={'query': tmp_query})
    json_data = json.loads(r.text)
    try:
      df_data = json_data['data']["tokenDayDatas"]
      df = pd.DataFrame(df_data)
      frames.append(df)
    except KeyError:
      break

  result = pd.concat(frames)
  result['date'] = pd.to_datetime(result['date'], unit='s')
  result[["volumeUSD","totalValueLockedUSD","feesUSD","priceUSD","open","high","low","close"]] = result[["volumeUSD","totalValueLockedUSD","feesUSD","priceUSD","open","high","low","close"]].apply(pd.to_numeric)
  result ['token_id'] = token_id
  return result


In [ ]:
USD_pd = Extracting_Token_day_data(token_usdc_id)
USD_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508 entries, 0 to 7
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 508 non-null    datetime64[ns]
 1   volumeUSD            508 non-null    float64       
 2   totalValueLockedUSD  508 non-null    float64       
 3   priceUSD             508 non-null    float64       
 4   feesUSD              508 non-null    float64       
 5   open                 508 non-null    float64       
 6   high                 508 non-null    float64       
 7   low                  508 non-null    float64       
 8   close                508 non-null    float64       
 9   token_id             508 non-null    object        
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 43.7+ KB


In [ ]:
ether_pd = Extracting_Token_day_data(token_weth_id)
ether_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508 entries, 0 to 7
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 508 non-null    datetime64[ns]
 1   volumeUSD            508 non-null    float64       
 2   totalValueLockedUSD  508 non-null    float64       
 3   priceUSD             508 non-null    float64       
 4   feesUSD              508 non-null    float64       
 5   open                 508 non-null    float64       
 6   high                 508 non-null    float64       
 7   low                  508 non-null    float64       
 8   close                508 non-null    float64       
 9   token_id             508 non-null    object        
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 43.7+ KB


In [ ]:
bit_pd = Extracting_Token_day_data(token_wbtc_id)
bit_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508 entries, 0 to 7
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 508 non-null    datetime64[ns]
 1   volumeUSD            508 non-null    float64       
 2   totalValueLockedUSD  508 non-null    float64       
 3   priceUSD             508 non-null    float64       
 4   feesUSD              508 non-null    float64       
 5   open                 508 non-null    float64       
 6   high                 508 non-null    float64       
 7   low                  508 non-null    float64       
 8   close                508 non-null    float64       
 9   token_id             508 non-null    object        
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 43.7+ KB


In [29]:
uni_pd = Extracting_Token_day_data(token_uni_id)
uni_pd.info()

NameError: ignored

In [ ]:
dai_pd = Extracting_Token_day_data(token_dai_id)
dai_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508 entries, 0 to 7
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 508 non-null    datetime64[ns]
 1   volumeUSD            508 non-null    float64       
 2   totalValueLockedUSD  508 non-null    float64       
 3   priceUSD             508 non-null    float64       
 4   feesUSD              508 non-null    float64       
 5   open                 508 non-null    float64       
 6   high                 508 non-null    float64       
 7   low                  508 non-null    float64       
 8   close                508 non-null    float64       
 9   token_id             508 non-null    object        
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 43.7+ KB


In [ ]:
result = pd.concat([dai_pd,uni_pd,bit_pd,ether_pd,USD_pd])
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2540 entries, 0 to 7
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   date                 2540 non-null   datetime64[ns]
 1   volumeUSD            2540 non-null   float64       
 2   totalValueLockedUSD  2540 non-null   float64       
 3   priceUSD             2540 non-null   float64       
 4   feesUSD              2540 non-null   float64       
 5   open                 2540 non-null   float64       
 6   high                 2540 non-null   float64       
 7   low                  2540 non-null   float64       
 8   close                2540 non-null   float64       
 9   token_id             2540 non-null   object        
dtypes: datetime64[ns](1), float64(8), object(1)
memory usage: 218.3+ KB


In [ ]:
result.head()

,date,volumeUSD,totalValueLockedUSD,priceUSD,feesUSD,open,high,low,close,token_id
0,2022-09-23,3.408121e+07,8.074883e+08,1.0,39890.651449,1.0,1.000556,0.999754,1.0,0x6b175474e89094c44da98b954eedeac495271d0f
1,2022-09-22,4.704904e+07,8.074820e+08,1.0,47731.178525,1.0,1.001763,0.999608,1.0,0x6b175474e89094c44da98b954eedeac495271d0f
2,2022-09-21,7.963730e+07,7.581289e+08,1.0,83007.763601,1.0,1.001563,0.998301,1.0,0x6b175474e89094c44da98b954eedeac495271d0f
3,2022-09-20,5.444140e+07,7.840144e+08,1.0,42722.396633,1.0,1.001037,0.999601,1.0,0x6b175474e89094c44da98b954eedeac495271d0f
4,2022-09-19,6.864848e+07,7.861788e+08,1.0,43798.212720,1.0,1.000530,0.999249,1.0,0x6b175474e89094c44da98b954eedeac495271d0f


In [ ]:
result.to_csv('TokenDayDatas.csv',index=False)

####Extracting Pools

In [ ]:
query = """
 query {
  pools (orderBy: totalValueLockedUSD , orderDirection: desc , skip : %d) {
    id
    totalValueLockedUSD
    token0 {
      symbol
    }
    token1 {
      symbol
    }
    
    feeTier
    volumeUSD
    liquidity
    token0Price
    token1Price
    volumeToken0
    volumeToken1
    feesUSD
		createdAtTimestamp
    createdAtBlockNumber
    totalValueLockedToken0
    totalValueLockedToken1
  
  }

}

"""


In [ ]:
 pool_df = Extract_data(query, "pools")
 pool_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 99
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      5000 non-null   object
 1   totalValueLockedUSD     5000 non-null   object
 2   token0                  5000 non-null   object
 3   token1                  5000 non-null   object
 4   feeTier                 5000 non-null   object
 5   volumeUSD               5000 non-null   object
 6   liquidity               5000 non-null   object
 7   token0Price             5000 non-null   object
 8   token1Price             5000 non-null   object
 9   volumeToken0            5000 non-null   object
 10  volumeToken1            5000 non-null   object
 11  feesUSD                 5000 non-null   object
 12  createdAtTimestamp      5000 non-null   object
 13  createdAtBlockNumber    5000 non-null   object
 14  totalValueLockedToken0  5000 non-null   object
 15  totalV

In [ ]:
for index, row in pool_df.iterrows():
  row["token0"] = row["token0"]["symbol"]
  row["token1"] = row["token1"]["symbol"] 

In [ ]:
pool_df.head()

,id,totalValueLockedUSD,token0,token1,feeTier,volumeUSD,liquidity,token0Price,token1Price,volumeToken0,volumeToken1,feesUSD,createdAtTimestamp,createdAtBlockNumber,totalValueLockedToken0,totalValueLockedToken1
0,0xa850478adaace4c08fc61de44d8cf3b64f359bec,12887625156.60443191907094774602519,UMIIE,UMIIE2,500,0,1706245281880037395956227425,0.01000586374960270227477201921193648,99.94139686738253752770898237370869,1000103.099007820197283854,99930988.660196871689931931,0,1625360210,12757982,50050480.103497613002904654,14644676667.240849617715772909
1,0x5777d92f208679db4b9778590fa3cab3ac9e2168,834206170.308562890847366528,DAI,USDC,100,11411607736.77493832205149262472364,4163037911750196992890830,0.99998302784338051475326900968195,1.000016972444678474545349895298435,11412168628.50957757957106843,11412326279.049196,1141160.773677493832205149262472364,1636771503,13605124,389285727.129007890847366528,444920443.179555
2,0x6c6bc977e13df9b0de53b251522280bb72383700,489041877.000607684411508241,DAI,USDC,500,6102830572.498203430944248000634074,485130580756061625850228,1.000050235423474734353489884721661,0.9999497670999962696978867270493328,6100232517.537320802884502008,6103596388.339653,3051415.286249101715472124000316969,1620158293,12369760,354477522.707567684411508241,134564354.29304
3,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,319772332.483559270671390910770411,USDC,WETH,3000,63017458482.66488360842323667584152,11068875157710941650,1290.325183053788392010240569154229,0.000774998436931470847893237114174133,63013401227.474753,26112440.252158879727053143,189052375.4479946508252697100275003,1620169800,12370624,148426123.099756,132793.044446580057440036
4,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,255242182.7426305631533303818730392,USDC,WETH,500,279292380212.9670687385341697513886,10281233307956748851,1289.964922767866667476498490907309,0.0007752148778234283917896513986537323,279268336587.364144,116270210.071247030562748386,139646190.1064835343692670848757122,1620250931,12376729,118317828.834612,106116.160256565918334734


In [ ]:
pool_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 99
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      5000 non-null   object
 1   totalValueLockedUSD     5000 non-null   object
 2   token0                  5000 non-null   object
 3   token1                  5000 non-null   object
 4   feeTier                 5000 non-null   object
 5   volumeUSD               5000 non-null   object
 6   liquidity               5000 non-null   object
 7   token0Price             5000 non-null   object
 8   token1Price             5000 non-null   object
 9   volumeToken0            5000 non-null   object
 10  volumeToken1            5000 non-null   object
 11  feesUSD                 5000 non-null   object
 12  createdAtTimestamp      5000 non-null   object
 13  createdAtBlockNumber    5000 non-null   object
 14  totalValueLockedToken0  5000 non-null   object
 15  totalV

In [ ]:
pool_df.to_csv('Pools.csv',index=False)

#### Extracting PoolDayDatas

In [ ]:
def Extracting_Pool_day_data (pool_id):
  # new URL / after the update 
  url = 'https://api.thegraph.com/subgraphs/name/ianlapham/v3-minimal'
  # old URL
  #url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
  frames = []
#   query = """query {
#   poolDayDatas (where : { pool : "%s"} , orderBy:date, orderDirection:desc,skip:%d)
#   {
#     date
#     liquidity
#     token0Price
#     token1Price
#     tvlUSD
#     volumeUSD
#     feesUSD
#     tick
    
#   }
# }"""

  query = """query {
  poolDayDatas (where : { pool : "%s"} , orderBy:date, orderDirection:desc,skip:%d)
  {
    date
    liquidity
    token0Price
    token1Price
    totalValueLockedUSD
    volumeUSD
    feesUSD
    tick
    
  }
}"""
  for i in range(0,5000,100):
    tmp_query = query%(pool_id,i)
    r = requests.post(url, json={'query': tmp_query})
    json_data = json.loads(r.text)
    try:
      df_data = json_data['data']["poolDayDatas"]
      df = pd.DataFrame(df_data)
      frames.append(df)
    except KeyError:
      break

  result = pd.concat(frames)
  result['date'] = pd.to_datetime(result['date'], unit='s')
  result['liquidity'] = result['liquidity'].astype(float)
  result[["liquidity","token0Price","token1Price","tvlUSD","volumeUSD","feesUSD","tick"]] = result[["liquidity","token0Price","token1Price","tvlUSD","volumeUSD","feesUSD","tick"]].apply(pd.to_numeric)
  result["Pool_ID"] = pool_id
  return result


In [ ]:
dai_usdc100_df = Extracting_Pool_day_data(dai_usdc100_id)
dai_usdc100_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 315 entries, 0 to 14
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         315 non-null    datetime64[ns]
 1   liquidity    315 non-null    float64       
 2   token0Price  315 non-null    float64       
 3   token1Price  315 non-null    float64       
 4   tvlUSD       315 non-null    float64       
 5   volumeUSD    315 non-null    float64       
 6   feesUSD      315 non-null    float64       
 7   tick         315 non-null    int64         
 8   Pool_ID      315 non-null    object        
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 24.6+ KB


In [ ]:
usdc_WETH_3000_df = Extracting_Pool_day_data(usdc_WETH_3000_id)
usdc_WETH_3000_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508 entries, 0 to 7
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         508 non-null    datetime64[ns]
 1   liquidity    508 non-null    float64       
 2   token0Price  508 non-null    float64       
 3   token1Price  508 non-null    float64       
 4   tvlUSD       508 non-null    float64       
 5   volumeUSD    508 non-null    float64       
 6   feesUSD      508 non-null    float64       
 7   tick         507 non-null    float64       
 8   Pool_ID      508 non-null    object        
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 39.7+ KB


In [ ]:
wbtc_WETH_3000_df = Extracting_Pool_day_data(wbtc_WETH_3000_id)
wbtc_WETH_3000_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508 entries, 0 to 7
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         508 non-null    datetime64[ns]
 1   liquidity    508 non-null    float64       
 2   token0Price  508 non-null    float64       
 3   token1Price  508 non-null    float64       
 4   tvlUSD       508 non-null    float64       
 5   volumeUSD    508 non-null    float64       
 6   feesUSD      508 non-null    float64       
 7   tick         508 non-null    int64         
 8   Pool_ID      508 non-null    object        
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 39.7+ KB


In [ ]:
UNI_WETH_3000_df = Extracting_Pool_day_data(UNI_WETH_3000_id)
UNI_WETH_3000_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 508 entries, 0 to 7
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         508 non-null    datetime64[ns]
 1   liquidity    508 non-null    float64       
 2   token0Price  508 non-null    float64       
 3   token1Price  508 non-null    float64       
 4   tvlUSD       508 non-null    float64       
 5   volumeUSD    508 non-null    float64       
 6   feesUSD      508 non-null    float64       
 7   tick         507 non-null    float64       
 8   Pool_ID      508 non-null    object        
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 39.7+ KB


In [ ]:
result = pd.concat([UNI_WETH_3000_df,wbtc_WETH_3000_df,usdc_WETH_3000_df,dai_usdc100_df])
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1839 entries, 0 to 14
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         1839 non-null   datetime64[ns]
 1   liquidity    1839 non-null   float64       
 2   token0Price  1839 non-null   float64       
 3   token1Price  1839 non-null   float64       
 4   tvlUSD       1839 non-null   float64       
 5   volumeUSD    1839 non-null   float64       
 6   feesUSD      1839 non-null   float64       
 7   tick         1837 non-null   float64       
 8   Pool_ID      1839 non-null   object        
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 143.7+ KB


In [ ]:
result.to_csv('PoolDayDatas.csv',index=False)

#### Extracting General Ticks



In [ ]:
query = """
 query { 
  ticks (skip : %d ,orderBy: createdAtTimestamp , orderDirection: desc)
  {
    id
    poolAddress
    tickIdx
    pool
    {
      id
    }
    liquidityGross
    liquidityNet
    price0
    price1
    createdAtTimestamp
    createdAtBlockNumber
  }
}

"""


In [ ]:
 tdf = Extract_data(query, "ticks")
 tdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 99
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    5000 non-null   object
 1   poolAddress           5000 non-null   object
 2   tickIdx               5000 non-null   object
 3   pool                  5000 non-null   object
 4   liquidityGross        5000 non-null   object
 5   liquidityNet          5000 non-null   object
 6   price0                5000 non-null   object
 7   price1                5000 non-null   object
 8   createdAtTimestamp    5000 non-null   object
 9   createdAtBlockNumber  5000 non-null   object
dtypes: object(10)
memory usage: 429.7+ KB


In [ ]:
for index, row in tdf.iterrows():
  row["pool"] = row["pool"]["id"] 

In [ ]:
tdf.head()

,id,poolAddress,tickIdx,pool,liquidityGross,liquidityNet,price0,price1,createdAtTimestamp,createdAtBlockNumber
0,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8#346380,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,346380,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,246162937765998607,246162937765998607,1102402424470636.927756250890122121,0.00000000000000090710976119286966376851998176...,1663935143,15595968
1,0x4bec87cb126de6c1f8b410e32d1f4ae472fdd83b#-20...,0x4bec87cb126de6c1f8b410e32d1f4ae472fdd83b,-201360,0x4bec87cb126de6c1f8b410e32d1f4ae472fdd83b,10609116095335470,-10609116095335470,0.000000001800874841400448002512182369404989,555285673.9463089434345479848637841,1663934891,15595947
2,0xf7849d0852fc588210b9c0d8b26f43c0c9bc1470#-13...,0xf7849d0852fc588210b9c0d8b26f43c0c9bc1470,-135000,0xf7849d0852fc588210b9c0d8b26f43c0c9bc1470,40947930414389079982241,40947930414389079982241,0.000001371884734429110014646117801662617,728924.2127299678227919792709308213,1663934735,15595934
3,0x9445bd19767f73dcae6f2de90e6cd31192f62589#277800,0x9445bd19767f73dcae6f2de90e6cd31192f62589,277800,0x9445bd19767f73dcae6f2de90e6cd31192f62589,57507473077847958,-57507473077847958,1159037566472.493357294958942234885,0.0000000000008627848043299227379650614160738691,1663934111,15595883
4,0xe42318ea3b998e8355a3da364eb9d48ec725eb45#43500,0xe42318ea3b998e8355a3da364eb9d48ec725eb45,43500,0xe42318ea3b998e8355a3da364eb9d48ec725eb45,392904121523638458835,-392904121523638458835,77.4616143151586962915846420080363,0.01290961993034925677034396037550325,1663933391,15595823


In [ ]:
tdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 99
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    5000 non-null   object
 1   poolAddress           5000 non-null   object
 2   tickIdx               5000 non-null   object
 3   pool                  5000 non-null   object
 4   liquidityGross        5000 non-null   object
 5   liquidityNet          5000 non-null   object
 6   price0                5000 non-null   object
 7   price1                5000 non-null   object
 8   createdAtTimestamp    5000 non-null   object
 9   createdAtBlockNumber  5000 non-null   object
dtypes: object(10)
memory usage: 429.7+ KB


In [ ]:
tdf.to_csv('General_Ticks.csv',index=False)

#### Extracting uniswapDayDates

In [5]:

query = """
 query {
  uniswapDayDatas (skip : %d , orderBy: date , orderDirection:desc)
  {
    id
    date
    volumeETH
    volumeUSD
    volumeUSDUntracked
    feesUSD
    txCount
    tvlUSD
  }
}
"""


In [6]:
 tdf = Extract_data(query, "uniswapDayDatas")
 tdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 510 entries, 0 to 9
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  510 non-null    object
 1   date                510 non-null    int64 
 2   volumeETH           510 non-null    object
 3   volumeUSD           510 non-null    object
 4   volumeUSDUntracked  510 non-null    object
 5   feesUSD             510 non-null    object
 6   txCount             510 non-null    object
 7   tvlUSD              510 non-null    object
dtypes: int64(1), object(7)
memory usage: 35.9+ KB


In [7]:
tdf.head()

,id,date,volumeETH,volumeUSD,volumeUSDUntracked,feesUSD,txCount,tvlUSD
0,19260,1664064000,41808.96984756841540054928614069313,55246253.10123311543487006944284265,0,64983.20993309552086495656747326355,20552279,10504721785.33476754884267158424524
1,19259,1663977600,411917.668920571420170830294365871,548059851.4982977532110679378439496,0,607927.8985586753817325945061931862,20544737,10445415098.16606953531631630606567
2,19258,1663891200,684975.3311218756633673456120045697,895732932.5320392183943394108234407,0,1196517.894051384910294894360261242,20497170,10500764534.69024577146600810209887
3,19257,1663804800,677680.2525415679185679451563958968,869577845.9258006669805956222923204,0,1122297.620916022671321777081253503,20434882,10490544001.69283982875454940389994
4,19256,1663718400,833642.4632467218565925091173449278,1100907057.866486558540979049417866,0,1502771.602234087556472982341012162,20380129,9906515571.516406653776277337647054


In [8]:
tdf.to_csv('uniswapDayDatas.csv',index=False)

dt_object = 2022-09-19 00:00:00
type(dt_object) = <class 'datetime.datetime'>


#### Extracting uniswapDayDates - fixed


In [16]:

query = """
 query {
  uniswapDayDatas (skip:%d , orderBy: date , orderDirection:desc)
  {
    id
    date
    volumeETH
    volumeUSD
    volumeUSDUntracked
    feesUSD
    txCount
    totalValueLockedUSD
  }
}
"""


In [17]:
 tdf = Extract_data(query, "uniswapDayDatas")
 tdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 510 entries, 0 to 9
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   510 non-null    object
 1   date                 510 non-null    int64 
 2   volumeETH            510 non-null    object
 3   volumeUSD            510 non-null    object
 4   volumeUSDUntracked   510 non-null    object
 5   feesUSD              510 non-null    object
 6   txCount              510 non-null    object
 7   totalValueLockedUSD  510 non-null    object
dtypes: int64(1), object(7)
memory usage: 35.9+ KB


In [18]:
tdf.head()

,id,date,volumeETH,volumeUSD,volumeUSDUntracked,feesUSD,txCount,totalValueLockedUSD
0,19260,1664064000,42969.11178118112967491057937144147,56784425.24970289075582547700084968,0,66661.03247213506486039249977025225,20552673,3779229052.854886037663166387200786
1,19259,1663977600,411919.5931978148803080630661541877,548062411.0703528567599294927653008,0,607928.8584299254464370943424747866,20544737,3764868751.944992447889140220913262
2,19258,1663891200,685005.6839536820886578551170207149,895772783.6724875607888060357901996,0,1196587.672238011722724074517554304,20497170,3768647803.232317866150488235632406
3,19257,1663804800,677703.8465984614700705322348582553,869608446.7074058154525101431451115,0,1122345.964436474178128231235991568,20434882,3747526534.756836015217784078925473
4,19256,1663718400,833675.3990540668625873681126664709,1100949362.682986266460852354188552,0,1502881.791985075263067507843469826,20380129,3560542287.328013172302979995699919


In [19]:
tdf.to_csv('uniswapDayDatas_fixed.csv',index=False)

dt_object = 2022-09-19 00:00:00
type(dt_object) = <class 'datetime.datetime'>


#### Extracting mints

In [ ]:
def Extracting_mints_bypoolid (pool_id):
  #old URL
  #url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
  # new URL / after the update 
  url = 'https://api.thegraph.com/subgraphs/name/ianlapham/v3-minimal'
  frames = []
  query = """query {
    mints ( where : {pool : "%s" , timestamp_gt : %d}, orderBy: timestamp, orderDirection: asc)
  {
    timestamp
    owner
    sender
    amountUSD
    amount0
    amount1
  }
  }"""
  ts = 0 
  while ( ts < 1663941971):
    #print("start")
    tmp_query = query%(pool_id,ts)
    r = requests.post(url, json={'query': tmp_query})
    json_data = json.loads(r.text)
    try:
      df_data = json_data['data']["mints"]
      df = pd.DataFrame(df_data)
      ts = int (df.iloc[-1:]["timestamp"])
      #print (ts)
      frames.append(df)
    except KeyError:
      break
    #print("end")
    

  result_mint = pd.concat(frames)
  result_mint['timestamp'] = pd.to_datetime(result_mint['timestamp'], unit='s')
  result_mint[["amountUSD","amount0","amount1"]] = result_mint[["amountUSD","amount0","amount1"]].apply(pd.to_numeric)
  result_mint['type'] = 'mint'
  result_mint['Pool'] = pool_id

  return result_mint

In [ ]:
dai_usdc100_mints = Extracting_mints_bypoolid(dai_usdc100_id)
dai_usdc100_mints.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2176 entries, 0 to 75
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  2176 non-null   datetime64[ns]
 1   owner      2176 non-null   object        
 2   sender     2176 non-null   object        
 3   amountUSD  2176 non-null   float64       
 4   amount0    2176 non-null   float64       
 5   amount1    2176 non-null   float64       
 6   type       2176 non-null   object        
 7   Pool       2176 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 153.0+ KB


In [ ]:
usdc_WETH_3000_mints = Extracting_mints_bypoolid(usdc_WETH_3000_id)
usdc_WETH_3000_mints.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70152 entries, 0 to 51
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  70152 non-null  datetime64[ns]
 1   owner      70152 non-null  object        
 2   sender     70152 non-null  object        
 3   amountUSD  70152 non-null  float64       
 4   amount0    70152 non-null  float64       
 5   amount1    70152 non-null  float64       
 6   type       70152 non-null  object        
 7   Pool       70152 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 4.8+ MB


In [ ]:
wbtc_WETH_3000_mints = Extracting_mints_bypoolid(wbtc_WETH_3000_id)
wbtc_WETH_3000_mints.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16159 entries, 0 to 58
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  16159 non-null  datetime64[ns]
 1   owner      16159 non-null  object        
 2   sender     16159 non-null  object        
 3   amountUSD  16159 non-null  float64       
 4   amount0    16159 non-null  float64       
 5   amount1    16159 non-null  float64       
 6   type       16159 non-null  object        
 7   Pool       16159 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 1.1+ MB


In [ ]:
uni_WETH_3000_mints = Extracting_mints_bypoolid(UNI_WETH_3000_id)
uni_WETH_3000_mints.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10798 entries, 0 to 97
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  10798 non-null  datetime64[ns]
 1   owner      10798 non-null  object        
 2   sender     10798 non-null  object        
 3   amountUSD  10798 non-null  float64       
 4   amount0    10798 non-null  float64       
 5   amount1    10798 non-null  float64       
 6   type       10798 non-null  object        
 7   Pool       10798 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 759.2+ KB


In [ ]:
result = pd.concat([uni_WETH_3000_mints,wbtc_WETH_3000_mints,usdc_WETH_3000_mints,dai_usdc100_mints])
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99285 entries, 0 to 75
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  99285 non-null  datetime64[ns]
 1   owner      99285 non-null  object        
 2   sender     99285 non-null  object        
 3   amountUSD  99285 non-null  float64       
 4   amount0    99285 non-null  float64       
 5   amount1    99285 non-null  float64       
 6   type       99285 non-null  object        
 7   Pool       99285 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 6.8+ MB


In [ ]:
result.head()

,timestamp,owner,sender,amountUSD,amount0,amount1,type,Pool
0,2021-05-04 19:52:36,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xc36442b4a4522e871399cd717abdd847ab11fe88,0.000000,1.00000,0.012644,mint,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801
1,2021-05-04 21:18:54,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xc36442b4a4522e871399cd717abdd847ab11fe88,0.000000,2.00000,0.014191,mint,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801
2,2021-05-05 05:07:53,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xc36442b4a4522e871399cd717abdd847ab11fe88,165.300088,14.04478,0.050000,mint,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801
3,2021-05-05 17:53:35,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xc36442b4a4522e871399cd717abdd847ab11fe88,3959.772614,90.00000,1.184419,mint,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801
4,2021-05-05 17:59:11,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xc36442b4a4522e871399cd717abdd847ab11fe88,0.410775,0.03000,0.000123,mint,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801


In [ ]:
result.to_csv("Mints.csv")

#### Extracting burns

In [ ]:
def Extracting_burns_bypoolid (pool_id):
  #old URL
  #url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
  # new URL / after the update 
  url = 'https://api.thegraph.com/subgraphs/name/ianlapham/v3-minimal'
  frames = []
  query = """query {
    burns ( where : {pool : "%s" , timestamp_gt : %d}, orderBy: timestamp, orderDirection: asc)
  {
    timestamp
    owner
    origin
    amountUSD
    amount0
    amount1
  }
  }"""
  ts = 0 
  while ( ts < 1663941971):
    #print("start")
    tmp_query = query%(pool_id,ts)
    r = requests.post(url, json={'query': tmp_query})
    json_data = json.loads(r.text)
    try:
      df_data = json_data['data']["burns"]
      df = pd.DataFrame(df_data)
      ts = int (df.iloc[-1:]["timestamp"])
      #print (ts)
      frames.append(df)
    except KeyError:
      break
    #print("end")
    

  result_mint = pd.concat(frames)
  result_mint['timestamp'] = pd.to_datetime(result_mint['timestamp'], unit='s')
  result_mint[["amountUSD","amount0","amount1"]] = result_mint[["amountUSD","amount0","amount1"]].apply(pd.to_numeric)
  result_mint['type'] = 'burn'
  result_mint['Pool'] = pool_id

  return result_mint

In [ ]:
dai_usdc100_burns = Extracting_burns_bypoolid(dai_usdc100_id)
dai_usdc100_burns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1400 entries, 0 to 99
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  1400 non-null   datetime64[ns]
 1   owner      1400 non-null   object        
 2   origin     1400 non-null   object        
 3   amountUSD  1400 non-null   float64       
 4   amount0    1400 non-null   float64       
 5   amount1    1400 non-null   float64       
 6   type       1400 non-null   object        
 7   Pool       1400 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 98.4+ KB


In [ ]:
usdc_WETH_3000_burns = Extracting_burns_bypoolid(usdc_WETH_3000_id)
usdc_WETH_3000_burns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65741 entries, 0 to 40
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  65741 non-null  datetime64[ns]
 1   owner      65741 non-null  object        
 2   origin     65741 non-null  object        
 3   amountUSD  65741 non-null  float64       
 4   amount0    65741 non-null  float64       
 5   amount1    65741 non-null  float64       
 6   type       65741 non-null  object        
 7   Pool       65741 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 4.5+ MB


In [ ]:
wbtc_WETH_3000_burns = Extracting_burns_bypoolid(wbtc_WETH_3000_id)
wbtc_WETH_3000_burns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19000 entries, 0 to 99
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  19000 non-null  datetime64[ns]
 1   owner      19000 non-null  object        
 2   origin     19000 non-null  object        
 3   amountUSD  19000 non-null  float64       
 4   amount0    19000 non-null  float64       
 5   amount1    19000 non-null  float64       
 6   type       19000 non-null  object        
 7   Pool       19000 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 1.3+ MB


In [ ]:
uni_WETH_3000_burns = Extracting_burns_bypoolid(UNI_WETH_3000_id)
uni_WETH_3000_burns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10736 entries, 0 to 35
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  10736 non-null  datetime64[ns]
 1   owner      10736 non-null  object        
 2   origin     10736 non-null  object        
 3   amountUSD  10736 non-null  float64       
 4   amount0    10736 non-null  float64       
 5   amount1    10736 non-null  float64       
 6   type       10736 non-null  object        
 7   Pool       10736 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 754.9+ KB


In [ ]:
result = pd.concat([uni_WETH_3000_burns,wbtc_WETH_3000_burns,usdc_WETH_3000_burns,dai_usdc100_burns])
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96877 entries, 0 to 99
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  96877 non-null  datetime64[ns]
 1   owner      96877 non-null  object        
 2   origin     96877 non-null  object        
 3   amountUSD  96877 non-null  float64       
 4   amount0    96877 non-null  float64       
 5   amount1    96877 non-null  float64       
 6   type       96877 non-null  object        
 7   Pool       96877 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 6.7+ MB


In [ ]:
result.head()

,timestamp,owner,origin,amountUSD,amount0,amount1,type,Pool
0,2021-05-05 20:33:42,0xc36442b4a4522e871399cd717abdd847ab11fe88,0xd753e7e9665aeca31a2c3d98accab5ccb1cc6afb,10225.762335,121.068093,1.498881,burn,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801
1,2021-05-05 20:46:23,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x224e69025a2f705c8f31efb6694398f8fd09ac5c,1362.460988,16.308053,0.195133,burn,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801
2,2021-05-05 20:46:23,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x224e69025a2f705c8f31efb6694398f8fd09ac5c,0.000000,0.000000,0.000000,burn,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801
3,2021-05-05 20:52:24,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x741a9c2287e3f91d5f5e57cd3d0ccde375219109,5835.852481,9.130709,1.574808,burn,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801
4,2021-05-05 20:52:24,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x741a9c2287e3f91d5f5e57cd3d0ccde375219109,0.000000,0.000000,0.000000,burn,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801


In [ ]:
result.to_csv("Burns.csv")

#### Extracting swaps

In [ ]:
def Extracting_swaps_bypoolid (pool_id):
  #old URL
  #url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
  # new URL / after the update 
  url = 'https://api.thegraph.com/subgraphs/name/ianlapham/v3-minimal'
  frames = []
  query = """query {
    swaps ( where : {pool : "%s" , timestamp_gt : %d}, orderBy: timestamp, orderDirection: asc)
  {
    timestamp
    sender
    recipient
    origin
    amountUSD
    amount0
    amount1
  }
  }"""
  ts = 0 
  while ( ts < 1663941971):
    #print("start")
    tmp_query = query%(pool_id,ts)
    r = requests.post(url, json={'query': tmp_query})
    json_data = json.loads(r.text)
    try:
      df_data = json_data['data']["swaps"]
      df = pd.DataFrame(df_data)
      ts = int (df.iloc[-1:]["timestamp"])
      #print (ts)
      frames.append(df)
    except KeyError:
      break
    #print("end")
    

  result_mint = pd.concat(frames)
  result_mint['timestamp'] = pd.to_datetime(result_mint['timestamp'], unit='s')
  result_mint[["amountUSD","amount0","amount1"]] = result_mint[["amountUSD","amount0","amount1"]].apply(pd.to_numeric)
  result_mint['type'] = 'swap'
  result_mint['Pool'] = pool_id

  return result_mint

In [ ]:
dai_usdc100_swaps = Extracting_swaps_bypoolid(dai_usdc100_id)
dai_usdc100_swaps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118000 entries, 0 to 99
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   timestamp  118000 non-null  datetime64[ns]
 1   sender     118000 non-null  object        
 2   recipient  118000 non-null  object        
 3   origin     118000 non-null  object        
 4   amountUSD  118000 non-null  float64       
 5   amount0    118000 non-null  float64       
 6   amount1    118000 non-null  float64       
 7   type       118000 non-null  object        
 8   Pool       118000 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 9.0+ MB


In [ ]:
usdc_WETH_3000_swaps = Extracting_swaps_bypoolid(usdc_WETH_3000_id)
usdc_WETH_3000_swaps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 334400 entries, 0 to 99
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   timestamp  334400 non-null  datetime64[ns]
 1   sender     334400 non-null  object        
 2   recipient  334400 non-null  object        
 3   origin     334400 non-null  object        
 4   amountUSD  334400 non-null  float64       
 5   amount0    334400 non-null  float64       
 6   amount1    334400 non-null  float64       
 7   type       334400 non-null  object        
 8   Pool       334400 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 25.5+ MB


In [ ]:
wbtc_WETH_3000_swaps = Extracting_swaps_bypoolid(wbtc_WETH_3000_id)
wbtc_WETH_3000_swaps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111300 entries, 0 to 99
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   timestamp  111300 non-null  datetime64[ns]
 1   sender     111300 non-null  object        
 2   recipient  111300 non-null  object        
 3   origin     111300 non-null  object        
 4   amountUSD  111300 non-null  float64       
 5   amount0    111300 non-null  float64       
 6   amount1    111300 non-null  float64       
 7   type       111300 non-null  object        
 8   Pool       111300 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 8.5+ MB


In [ ]:
uni_WETH_3000_swaps = Extracting_swaps_bypoolid(UNI_WETH_3000_id)
uni_WETH_3000_swaps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80300 entries, 0 to 99
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  80300 non-null  datetime64[ns]
 1   sender     80300 non-null  object        
 2   recipient  80300 non-null  object        
 3   origin     80300 non-null  object        
 4   amountUSD  80300 non-null  float64       
 5   amount0    80300 non-null  float64       
 6   amount1    80300 non-null  float64       
 7   type       80300 non-null  object        
 8   Pool       80300 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 6.1+ MB


In [ ]:
result = pd.concat([uni_WETH_3000_swaps,wbtc_WETH_3000_swaps,usdc_WETH_3000_swaps,dai_usdc100_swaps])
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 644000 entries, 0 to 99
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   timestamp  644000 non-null  datetime64[ns]
 1   sender     644000 non-null  object        
 2   recipient  644000 non-null  object        
 3   origin     644000 non-null  object        
 4   amountUSD  644000 non-null  float64       
 5   amount0    644000 non-null  float64       
 6   amount1    644000 non-null  float64       
 7   type       644000 non-null  object        
 8   Pool       644000 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 49.1+ MB


In [ ]:
result.head()

,timestamp,sender,recipient,origin,amountUSD,amount0,amount1,type,Pool
0,2021-05-05 08:35:47,0xe592427a0aece92de3edee1f18e0157c05861564,0x0459b3fbf7c1840ee03a63ca4aa95de48322322e,0x0459b3fbf7c1840ee03a63ca4aa95de48322322e,129.289579,-3.000000,0.039160,swap,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801
1,2021-05-05 18:03:43,0xe592427a0aece92de3edee1f18e0157c05861564,0x12696ad7ea03474ed5d2d2909c52f3d1d997c81e,0x12696ad7ea03474ed5d2d2909c52f3d1d997c81e,0.033432,-0.000747,0.000010,swap,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801
2,2021-05-05 18:08:16,0xe592427a0aece92de3edee1f18e0157c05861564,0xe592427a0aece92de3edee1f18e0157c05861564,0x5337122c6b5ce24d970ce771510d22aeaf038c44,44.465241,1.000000,-0.013300,swap,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801
3,2021-05-05 18:08:19,0xe592427a0aece92de3edee1f18e0157c05861564,0x22f9dcf4647084d6c31b2765f6910cd85c178c18,0x5097f85f9312d5e2ffd16c3668a63a623221f16e,6020.002910,135.546471,-1.800660,swap,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801
4,2021-05-05 18:08:55,0xe592427a0aece92de3edee1f18e0157c05861564,0xe8c73aa629605202d77ff31eedd55e92ebd3aefa,0xe8c73aa629605202d77ff31eedd55e92ebd3aefa,45.765859,-1.025484,0.013689,swap,0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801


In [ ]:
result.to_csv("Swaps.csv")

#### Extracting Positions - no updated version

In [ ]:
query = """
 query {
  positions (skip : %d ,orderBy: liquidity , orderDirection: desc)
  {
    id
    owner
    pool
    {
      id
    }
    token0
    {
      id
    }
    token1
    {
      id
    }
    tickLower
    {
      id
    }
    tickUpper
    {
      id
    }
    liquidity
    depositedToken0
    depositedToken1
    withdrawnToken0
    withdrawnToken1
    collectedFeesToken0
    collectedFeesToken1
    transaction
    {
      id
    }
    feeGrowthInside0LastX128
    feeGrowthInside1LastX128
    
  }
}
"""


In [ ]:
 tdf = Extract_data(query, "positions")
 tdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 99
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        5000 non-null   object
 1   owner                     5000 non-null   object
 2   pool                      5000 non-null   object
 3   token0                    5000 non-null   object
 4   token1                    5000 non-null   object
 5   tickLower                 5000 non-null   object
 6   tickUpper                 5000 non-null   object
 7   liquidity                 5000 non-null   object
 8   depositedToken0           5000 non-null   object
 9   depositedToken1           5000 non-null   object
 10  withdrawnToken0           5000 non-null   object
 11  withdrawnToken1           5000 non-null   object
 12  collectedFeesToken0       5000 non-null   object
 13  collectedFeesToken1       5000 non-null   object
 14  transaction               

In [ ]:
tdf.head()

,id,owner,pool,token0,token1,tickLower,tickUpper,liquidity,depositedToken0,depositedToken1,withdrawnToken0,withdrawnToken1,collectedFeesToken0,collectedFeesToken1,transaction,feeGrowthInside0LastX128,feeGrowthInside1LastX128
0,114444,0xabb9d90f78b55013c5cea9e91c9419ddff4fc0ac,{'id': '0xecdcb160c0fa92ee6430972b8e918c1b073e...,{'id': '0x7b93af056ecac7ba97a5f669cf127f9257e4...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,{'id': '0xecdcb160c0fa92ee6430972b8e918c1b073e...,{'id': '0xecdcb160c0fa92ee6430972b8e918c1b073e...,11114450057194205771809438988582841,10000000000000000.0000000000000001,0,0,0,0,0,{'id': '0xb1980e16d55c2036cdb25eb52b667a5c594c...,0,0
1,40589,0x7d5f6049014e67f16c16335374d65370fcba34e1,{'id': '0x79c9946b4cb90bd21fbe330110f5658f9d35...,{'id': '0x926a2494acba43fcc2442fda80a1cbaf99ed...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,{'id': '0x79c9946b4cb90bd21fbe330110f5658f9d35...,{'id': '0x79c9946b4cb90bd21fbe330110f5658f9d35...,10459723782139312351123351992955751,59999996500596.466897904020453762,0,0,0,0,0,{'id': '0x8e770f40fba7bf53b252f4c9765082835197...,0,0
2,41261,0xbd2f670afcaf98b939203173a4370d71c75963f8,{'id': '0x000f0c0b0b791e855dcc5ad6501c7529dea8...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,{'id': '0xd1063ee5ec2891991a29fefb52bcc448cd38...,{'id': '0x000f0c0b0b791e855dcc5ad6501c7529dea8...,{'id': '0x000f0c0b0b791e855dcc5ad6501c7529dea8...,2032209333937888030579013479745504,0,10000000000,0,0,0,0,{'id': '0xca40c78f8951b97d9f7a5e6eb3d91f30fb45...,0,0
3,65266,0x0797a0bd634e0949be80d49b4bc3e7225f02d747,{'id': '0x68f5d4009f9001c8d00ced56c1e90702c998...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,{'id': '0xdba6c02329d5f84856b32f401d4c5e3009e7...,{'id': '0x68f5d4009f9001c8d00ced56c1e90702c998...,{'id': '0x68f5d4009f9001c8d00ced56c1e90702c998...,1907855112094752544864253485599904,0,36948200048614,0,0,0,0,{'id': '0x6bf4d1b7c8fc2d2d8bba12b6e4fb351a6bb2...,0,0
4,79240,0x06773ae9559a0e0c8dba89d3cd4f7eac74b225b8,{'id': '0x596101125094f9d65a579b2cfc4c26059cb6...,{'id': '0x78a52e12c7b63d05c12f9608307587cf654e...,{'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c75...,{'id': '0x596101125094f9d65a579b2cfc4c26059cb6...,{'id': '0x596101125094f9d65a579b2cfc4c26059cb6...,1061529810167122922185668885105454,400000000,0,0,0,0,0,{'id': '0x840f2f9511a640b45ef979113b3f313963ff...,0,0


In [ ]:
for index, row in tdf.iterrows():
  row["pool"] = row["pool"]["id"]
  row["token0"] = row["token0"]["id"]
  row["token1"] = row["token1"]["id"]
  row["tickLower"] = row["tickLower"]["id"]
  row["tickUpper"] = row["tickUpper"]["id"]
  row["transaction"] = row["transaction"]["id"]

In [ ]:
tdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 99
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        5000 non-null   object
 1   owner                     5000 non-null   object
 2   pool                      5000 non-null   object
 3   token0                    5000 non-null   object
 4   token1                    5000 non-null   object
 5   tickLower                 5000 non-null   object
 6   tickUpper                 5000 non-null   object
 7   liquidity                 5000 non-null   object
 8   depositedToken0           5000 non-null   object
 9   depositedToken1           5000 non-null   object
 10  withdrawnToken0           5000 non-null   object
 11  withdrawnToken1           5000 non-null   object
 12  collectedFeesToken0       5000 non-null   object
 13  collectedFeesToken1       5000 non-null   object
 14  transaction               

In [ ]:
tdf.to_csv('Positions.csv',index=False)